In [1]:
%matplotlib inline

import graph_tool as gt
from graph_tool.draw import *
import graph_tool.collection as collection
import graph_tool.all.minimize_nested_blockmodel_dl as minimize_nested_blockmodel_dl
import graph_tool.generation as gen
import axelrod as axl
import pandas as pd
import toolz as tz
import numpy as np  
import matplotlib.pyplot as plt
from random import choice, sample, random
from copy import deepcopy
from collections import Counter
from time import time
from math import floor, sqrt, inf
from itertools import product, combinations
import seaborn as sns
import asyncio
import ipywidgets
 

In [2]:
from time import time

In [3]:
from collections.abc import Iterable
def flatten(x):
    if isinstance(x, Iterable):
        return sum([flatten(i) for i in x], [])
    return [x]

In [4]:
# project params
outputPath = './sf.png'
DEBUG=False

'''Utils'''
def transposeList(l) = list(map(list, zip(*l)))
def timeFn(text, fn):
    start = time()
    res = fn()
    end = time()
    if DEBUG:
        print(text + f' took {end-start}s')
    return res
def orderTsMatrixPlot(ts, M) = [y for x in  [ts[(M-1-i)::M] for i in range(M)] for y in x] 
    

In [5]:
# network params
_k = 30
N = _N = 500
_m = 2
_c = 0 
_gamma = 1

# game params
_R=1
_P=0
_T=2 # T€[0,2] 
_S=-1 # S€[-1,1]
def genTSParams(M):
    t_ = np.linspace(0,2,M)
    s_ = np.linspace(-1,1,M)
    _ts = product(t_, s_)|>list
    return _ts

# evo params
_episode_n = 100
_te = 1
_ta = 0.5
_W = _te/_ta
_W2 = 0
_beta = 0.001

In [6]:
'''Init''' 
C = 'C'
D = 'D'
def initStrats(N) = np.random.choice([C,D],N)
def initPayoffs(N) = np.zeros(N)

'''Graph'''
# returns the number k
def uniform(k):
    return k
# returns a number betwee 1 and k
def randSample(max):
    accept = False
    while not accept:
        k = np.random.randint(1,max+1)
        accept = np.random.random() < 1.0/k
    return k
# returns a new scale free graph
def initScaleFreeGraph(N=_N, m=_m, c=_c, gamma=_gamma) = gen.price_network(N=N, m=m, c=c, gamma=gamma, directed=False)
# returns a new random graph
def initRandomGraph(N=_N, k=_k) = gt.generation.random_graph(N, x->randSample(k), directed=False)
# returns a new random graph with a uniform degree distribution
def initUniformRandomGraph(N=_N, k=30) = gt.generation.random_graph(N, x->uniform(k), directed=False)
# sample one neighbor of node id in a graph, return none if no neighbors


In [7]:
# get first degree neighbors of the nodes ids
def getNeighbors(graph, ids):
    return flatten([graph.get_all_neighbors(id) for id in ids]) |> set |> list
# get a list of for the nth degree neighbors between 0 and n
def getNthNeighbors(graph, id, n):
    nthNeighbors = [[id]] + [[] for i in range(n)] 
    for i in range(n):
        nthNeighbors[i+1] = getNeighbors(graph, nthNeighbors[i])
    return nthNeighbors

In [8]:
def sampleStrat(strats, strat) = enumerate(strats) |> filter$(x-> x[1] == strat) |> map$(.[0]) |> list |> eligible -> None if not eligible else (np.random.choice(eligible) )


def isFinished(graph, triedList):
    return len(triedList) == graph.num_vertices()
# sample neighbors of x filtered by a function prioritizing closest neighbors and moving outward in rings if none available
def filterEgoSample(graph, filterFn, ids):
#     print(f"filterEgoSample: {ids}")
    neighbors = getNeighbors(graph, ids)
#     print(f"filterEgoSample neighbors: {neighbors}")
    elligibleNeighbors = filter(filterFn, neighbors) |> list
#     print(f"filterEgoSample eligible: {elligibleNeighbors}")
    if not elligibleNeighbors:
        if isFinished(graph, neighbors):
            return None
        return filterEgoSample(graph, filterFn, neighbors)
    return np.random.choice(elligibleNeighbors)


In [9]:
# exclude first neighbors
def filterEgoSampleUnique(graph, filterFn, id):
    excludeFirstNeighbors = x -> x not in ([id] + list(graph.get_all_neighbors(id)))
    return filterEgoSample(graph, x -> excludeFirstNeighbors(x) and filterFn(x), [id])

In [10]:
def sampleEgoStrat(graph, strats, strat, id):
    return filterEgoSampleUnique(graph, x->strats[x]==strat, id)

In [43]:
''' Neighbor Sampling'''
def sampleNeighbor(graph, id):
    neighbors = graph.get_all_neighbors(id)
    try:
        neigh = np.random.choice(neighbors) 
        return neigh
    except:
        return None
# sample one neighbor of node id1 in a graph, excluding id2 and its neighbors 
def sampleNeighborUnique(graph, id1, id2):
     neighbors = list(set(graph.get_all_neighbors(id1)) - set(graph.get_all_neighbors(id2)) - set([id2]))
     return np.random.choice(neighbors) if neighbors else None 
# sample any second degree neighbor of node id by random sampling a firt degree neighbor and then sampling its neighbors
def sampleSecondNeighbor(graph, id):
    firstNeighbor = sampleNeighbor(graph, id)
    secondNeighbor = sampleNeighbor(graph, firstNeighbor)
    return secondNeighbor
# sample any second degree neighbor of node id excluding id's first degree neighbors
def sampleSecondNeighborUnique(graph, id):
    firstNeighbors = graph.get_all_neighbors(id)
    for fN in np.random.shuffle(firstNeighbors):
        secondNeighbor = sampleNeighborUnique(graph, fN, id)
        if secondNeighbor:
            return secondNeighbor
    return None 
#checks if node has only 1 edge
def isLonely(graph, x) = graph.vertex(x).out_degree() == 1 
# SIDE EFFECT: replaces the graph's edge (x,y) with edge (x,z)
def rewireEdge(graph, x, y, z):
    graph.remove_edge(graph.edge(x,y))
    graph.add_edge(x, z)
    return graph
# neighbors of b eligible for rewiring a to 
def eligibleNewFriends(graph, b, a) = set(graph.get_all_neighbors(b)) - set(graph.get_all_neighbors(a)) - set([a]) |> list
# samples a node of a given strat from the list strats, returns its id
def sampleStrat(strats, strat) = enumerate(strats) |> filter$(x-> x[1] == strat) |> map$(.[0]) |> list |> eligible -> None if not eligible else (np.random.choice(eligible) )
def sampleStratEligible(graph, strats, strat, x):
    ofStrat = enumerate(strats) |> filter$(x-> x[1] == strat) |> map$(.[0]) |> list
    if not ofStrat:
        return None 
    else:
        eligible = set(ofStrat) - set(graph.get_all_neighbors(x)) - set([x]) |> list
        if not eligible:
            return None
        return np.random.choice(eligible)


In [12]:
'''Games'''
# Dictionary of game weights given T and S. R=1, P=0
def makeWeightDict(t,s) = {'R':_R, 'P':_P, 'T':t, 'S':s}
# Returns a dictionary that maps the strategies played to payoffs received for a set of 4 payoff metric
def makeDilemma(R=_R, P=_P, T=_T, S=_S) = {C:{C:[R,R], D:[S,T]}, D:{C:[T,S], D:[P,P]}} # T€[0,2] S€[-1,1]
# Returns a dictionary that maps the strategies played to payoffs received for T and S (canonical form of a 2x2)
def makeTSDilemma(t,s) = makeDilemma(**makeWeightDict(t,s))
dilemma = makeDilemma()
# Returns the payoff as a function of the dilemma, the ids and strats of the players
# playDilemma :: [strat] -> id -> id -> float
def playDilemma(dilemma, strats, id1, id2) = dilemma[strats[id1]][strats[id2]]
# cumulativePayoffs :: graph -> [strat] # Cumulative Payoff of 1 round of a node playing all its neighbors
def nodeCumPayoffs(dilemma, graph, strats, x):
    try:
        payoffs = [playDilemma(dilemma, strats, x, y)[0] for y in graph.get_all_neighbors(x)]
        return sum(payoffs)
    except:
        return 0
# Pair of cumulative payoffs 
def pairCumPayoffs(dilemma, graph, strats, a, b):
    return [nodeCumPayoffs(dilemma, graph, strats, a), nodeCumPayoffs(dilemma, graph, strats, b)]
        

In [13]:
'''Mediators'''
# No mediator, recommends a neighbor of y 
def useNoMed(graph, strats, y, x):
    eligible = eligibleNewFriends(graph, y, x)
    return None if not eligible else np.random.choice(eligible)

# Recommends a random cooperator
def useGoodMed(graph, strats, y, x):
    z = sampleStratEligible(graph, strats, C, x)
    return None if not z else z

# Recommends a random defector
def useBadMed(graph, strats, y, x):
    z = sampleStratEligible(graph, strats, D, x)
    return None if not z else z

# Recommends a random node
def useRandomMed(graph, strats, y, x):
    return np.random.choice(list(set(graph.get_vertices()) - set(graph.get_all_neighbors(x)) - set([x])) )

# Recs D to D and C to C
def useFairMed(graph, strats, y, x):
    z = sampleStratEligible(graph, strats, strats[int(x)], x)
    return None if not z else z


In [14]:
''' Ego mediators '''
def useLocalGoodMed(graph, strats, y, x) = sampleEgoStrat(graph, strats, C, x)
def useLocalBadMed(graph, strats, y, x) = sampleEgoStrat(graph, strats, D, x)
def useLocalRandomMed(graph, strats, y, x) = filterEgoSampleUnique(graph, x->True, x)
def useLocalFairMed(graph, strats, y, x) = sampleEgoStrat(graph, strats, strats[x], x)


In [15]:

'''Mediator strats'''
GOOD_MED = 'GOOD_MED'
BAD_MED = 'BAD_MED'
NO_MED = 'NO_MED'
RANDOM_MED = 'RANDOM_MED'
FAIR_MED = 'FAIR_MED'

_medSet = [NO_MED, GOOD_MED, FAIR_MED, RANDOM_MED, BAD_MED]
def initMedStrats(N, medSet) = [np.random.choice(medSet) for i in range(N)]
medDict = {NO_MED:useNoMed, GOOD_MED:useGoodMed, BAD_MED:useBadMed, RANDOM_MED:useRandomMed, FAIR_MED:useFairMed}
def useMed(medStrat, graph, strats, medStrats, y, x) = medDict[medStrat](graph, strats, y, x)

In [16]:
'''Structural Update'''
# Makes a tie update object
def TieUpdate(x,y,z) = {"updateType":"rewire", "x":x, "old":y, "new":z}
# Asks for a recommendation and keeps the same tie if there are no other valid nodes 
def decideRewire(graph, strats, x, y, medStrats):
    if isLonely(graph, y): 
        return TieUpdate(x,y,y) #enforcing graph connectedness
    z = useMed(medStrats[x], graph, strats, medStrats, y, x)
    if not z:
        return TieUpdate(x,y,y) #enforcing graph connectedness
    return TieUpdate(x,y,z)
# Decides whether a rewire should happen based on x and y's strats. If x is satisfied, nothing happens
def calcStructuralUpdate(beta, graph, dilemma, strats, _x, _y, p, medStrats):
    x,y = int(_x), int(_y)
    if (strats[x] == C and strats[y] == D):
        doRewire = random() < p        
        if doRewire:
            return decideRewire(graph, strats, x, y, medStrats)
        else:
            return TieUpdate(x,y,y)
    elif (strats[x] == D and strats[y] == D):
        keepX = random() < p
        args = [x,y] if keepX else [y,x]
        return decideRewire(graph, strats, args[0], args[1], medStrats)
    return TieUpdate(x,y,y)
# Applies a tie update to the graph
def updateTies(graph, tieUpdate):
    return rewireEdge(graph, tieUpdate["x"], tieUpdate["old"], tieUpdate["new"])
    
'''Strategy Evolution'''
# fermi formula, pairwise comparison
def fermi(beta, fitness_diff):
    return 1. / (1. + np.exp(-1 * beta * np.clip(fitness_diff, 0,None), dtype=np.float64))
# 
# def calcK(graph, x, y) = max(graph.vertex(x).out_degree(), graph.vertex(y).out_degree())
# def calcD(T=_T, S=_S) = max(T, 1) - min(S, 0)
# def transProb(calcK, P, x, y) = (P[y] - P[x]) / (calcK(x, y) * calcD())

# Returns a strategy update object
# updateStrat :: graph -> [strat] -> [float] -> id -> strat
def calcStrategyUpdate(beta, graph, dilemma, strats, x, y, p):
    doChangeStrat = random() < p  
    new = strats[y] if doChangeStrat else strats[x]
    return {"updateType":"strat", "x":x, "old":strats[x], "new":new}
# Applies a strategy update to the graph
def updateStrat(strats, update):
    strats[update["x"]] = update["new"]
    return strats

'''Mediator Evolution/Competition'''
def calcMedUpdate(beta, graph, dilemma, medStrats, x, y, p):
    doChangeStrat = random() < p  
    new = medStrats[y] if doChangeStrat else medStrats[x]
    return {"updateType":"mediator", "x":x, "old":medStrats[x], "new":new}
def updateMed(medStrats, update):
    medStrats[update["x"]] = update["new"]
    return medStrats

In [35]:
%load_ext cython

In [17]:
%%cython
import numpy as np
#cython optimized fermi
def fermi(double beta, int fitness_diff):
    cdef double exponent = -1 * beta * np.clip(fitness_diff, 0,None)
    cdef double denominator = (1. + np.exp(exponent, dtype=np.float64))
    cdef double result = 1. / denominator
    return result

UsageError: Cell magic `%%cython` not found.


In [96]:
%lprun -f fermi fermi(np.random.random(), np.random.randint(10))

/home/genesis/miniconda3/envs/gt/lib/python3.8/site-packages/line_profiler/line_profiler.py:328: UserWarning: Could not extract a code object for the object <built-in function fermi>
  profile = LineProfiler(*funcs)


In [44]:
%%cython 

# W2 is the time ratio between med update and (rewire/strat update). W2=0 means there are no med updates
def cy_runEvolutionCompetitionEp(N, beta, W, W2, dilemma, graph, medStrats, strats, history, _x, saveHistory=False):
    _y = sampleNeighbor(graph, _x)
    x,y = int(_x), int(_y)
    px, py = pairCumPayoffs(dilemma, graph, strats, x, y)
    p = fermi(beta, py - px)      
    doMedUpdate = np.random.rand() * (1+W2) > 1
    if doMedUpdate:
        medUpdate = calcMedUpdate(beta, graph, dilemma, medStrats, x, y, p)
        if saveHistory:
            history.append(medUpdate)
        medStrats = updateMed(medStrats, medUpdate)
    else:
        doStratUpdate = np.random.rand() * (1+W) <= 1
        if doStratUpdate:
            stratUpdate = calcStrategyUpdate(beta, graph, dilemma, strats, x, y, p)
            if saveHistory:
                history.append(stratUpdate)
            strats = updateStrat(strats, stratUpdate)
        else:
            graphUpdate = calcStructuralUpdate(beta, graph, dilemma, strats, _x, _y, p, medStrats)
            if saveHistory:
                history.append(graphUpdate)
            graph = updateTies(graph, graphUpdate)
    return strats, graph, history

def cy_genericRunEvolution(runEvolutionEp, N, episode_n, W, W2, dilemma, medStrats, strats, beta = 0.001, _graph=None, k=None, history=None, saveHistory=False):
    history = [] if history == None else history
    print(f"starting runEvolution, history len= {len(history)}, {'saving history, ' if saveHistory else ''} N={N}, episode_n={episode_n}")
    initialStrats = deepcopy(strats)
    initialMedStrats = deepcopy(medStrats)
    totalPayoffs = initPayoffs(N)
#     history = [] #[{updateType: "strat", x: N, old: {C,D}, new: {C,D}}, {updateType: "rewire", xy: (x,y), old: (x,y), new: (x,z)}]
    graph = deepcopy(_graph) if _graph else initUniformRandomGraph(N=N, k=(k if k else _k))
    for i in range(episode_n):
        x = np.random.randint(N)
        strats, graph, history = runEvolutionEp(N, beta, W, W2, dilemma, graph, medStrats, strats, history, x, saveHistory)
        if i % 5000 == 0:
            medEvoDone = any([x==N for x in Counter(medStrats).values()])
            stratEvoDone = any([x==N for x in Counter(strats).values()])
            if medEvoDone and stratEvoDone: 
                break
            if W2 == inf and medEvoDone:
                break
            if W == 0 and stratsEvoDone:
                break
    return {"graph":graph, "history":pd.DataFrame(history, dtype="category"), "initStrats":initialStrats, "finalStrats":deepcopy(strats), "initMedStrats":initialMedStrats,  "medStrats":deepcopy(medStrats)}



Error compiling Cython file:
------------------------------------------------------------
...

# W2 is the time ratio between med update and (rewire/strat update). W2=0 means there are no med updates
def cy_runEvolutionCompetitionEp(N, beta, W, W2, dilemma, graph, medStrats, strats, history, _x, saveHistory=False):
    _y = sampleNeighbor(graph, _x)
        ^
------------------------------------------------------------

/home/genesis/.cache/ipython/cython/_cython_magic_a6fd6172d249ce8bfbb24e84332d6b71.pyx:4:9: undeclared name not builtin: sampleNeighbor

Error compiling Cython file:
------------------------------------------------------------
...

# W2 is the time ratio between med update and (rewire/strat update). W2=0 means there are no med updates
def cy_runEvolutionCompetitionEp(N, beta, W, W2, dilemma, graph, medStrats, strats, history, _x, saveHistory=False):
    _y = sampleNeighbor(graph, _x)
    x,y = int(_x), int(_y)
    px, py = pairCumPayoffs(dilemma, graph, strats, x, y)
 

In [29]:
medSet=[NO_MED, GOOD_MED, BAD_MED]
k=30
N=500
beta=0.005
x=np.random.randint(N-1)
W=1
W2=1
strats = initStrats(N)
medStrats = initMedStrats(N,medSet)
dilemma = makeTSDilemma(2,-1)
graph=initUniformRandomGraph(N=N, k=k)
epArgs = {"N":N, "beta":beta, "W":W, "W2":W2, "dilemma":dilemma, "graph":graph, "medStrats":medStrats, "strats": strats, "history":[], "_x":x,}

In [40]:
episode_n=10000
evoArgs = {"runEvolutionEp":runEvolutionCompetitionEp,"N":N, "episode_n":episode_n, "beta":beta, "W":W, "W2":W2, "dilemma":dilemma, "_graph":graph, "medStrats":medStrats, "strats": strats, "history":[]}

In [114]:
%lprun -f runEvolutionCompetitionEp runEvolutionCompetitionEp(**epArgs)

In [41]:
%lprun -f genericRunEvolution genericRunEvolution(**evoArgs)

starting runEvolution, history len= 0,  N=500, episode_n=10000


In [ ]:
%lprun -f runEvolutionCompetitionEp runEvolutionCompetitionEp(**epArgs)

In [40]:
%%cython

def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum


In [18]:
# Runs a single evolution episode: either a strategy or a structural update to node x, based the timescale ratio W
def runBaseEvolutionEp(N, beta, W, dilemma, graph, strats, history, _x, useMediator, saveHistory=False):
    _y = sampleNeighbor(graph, _x)
    x,y = int(_x), int(_y)
    px, py = pairCumPayoffs(dilemma, graph, strats, x, y)
#     print(f"x {x}({strats[x]}, {px}), y {y}({strats[y]}, {py})")
    p = fermi(beta, py - px)      
    random_number = np.random.rand() * (1+W)
    doStratUpdate = random_number <= 1
    if doStratUpdate:
        stratUpdate = calcStrategyUpdate(beta, graph, dilemma, strats, x, y, p)
        if saveHistory:
            history.append(stratUpdate)
        strats = updateStrat(strats, stratUpdate)
    else:
        graphUpdate = calcStructuralUpdate(beta, graph, dilemma, strats, _x, _y, p, useMediator)
        if saveHistory:
            history.append(graphUpdate)
        graph = updateTies(graph, graphUpdate)
    return strats, graph, history

# W2 is the time ratio between med update and (rewire/strat update). W2=0 means there are no med updates
def runEvolutionCompetitionEp(N, beta, W, W2, dilemma, graph, medStrats, strats, history, _x, saveHistory=False):
    _y = sampleNeighbor(graph, _x)
    x,y = int(_x), int(_y)
    px, py = pairCumPayoffs(dilemma, graph, strats, x, y)
    p = fermi(beta, py - px)      
    doMedUpdate = np.random.rand() * (1+W2) > 1
    if doMedUpdate:
        medUpdate = calcMedUpdate(beta, graph, dilemma, medStrats, x, y, p)
        if saveHistory:
            history.append(medUpdate)
        medStrats = updateMed(medStrats, medUpdate)
    else:
        doStratUpdate = np.random.rand() * (1+W) <= 1
        if doStratUpdate:
            stratUpdate = calcStrategyUpdate(beta, graph, dilemma, strats, x, y, p)
            if saveHistory:
                history.append(stratUpdate)
            strats = updateStrat(strats, stratUpdate)
        else:
            graphUpdate = calcStructuralUpdate(beta, graph, dilemma, strats, _x, _y, p, medStrats)
            if saveHistory:
                history.append(graphUpdate)
            graph = updateTies(graph, graphUpdate)
    return strats, graph, history

def genericRunEvolution(runEvolutionEp, N, episode_n, W, W2, dilemma, medStrats, strats, beta = 0.001, _graph=None, k=None, history=None, saveHistory=False):
    history = [] if history == None else history
    print(f"starting runEvolution, history len= {len(history)}, {'saving history, ' if saveHistory else ''} N={N}, episode_n={episode_n}")
    initialStrats = deepcopy(strats)
    initialMedStrats = deepcopy(medStrats)
    totalPayoffs = initPayoffs(N)
#     history = [] #[{updateType: "strat", x: N, old: {C,D}, new: {C,D}}, {updateType: "rewire", xy: (x,y), old: (x,y), new: (x,z)}]
    graph = deepcopy(_graph) if _graph else initUniformRandomGraph(N=N, k=(k if k else _k))
    for i in range(episode_n):
        x = np.random.randint(N)
        strats, graph, history = runEvolutionEp(N, beta, W, W2, dilemma, graph, medStrats, strats, history, x, saveHistory)
        if i % 5000 == 0:
            medEvoDone = any([x==N for x in Counter(medStrats).values()])
            stratEvoDone = any([x==N for x in Counter(strats).values()])
            if medEvoDone and stratEvoDone: 
                break
            if W2 == inf and medEvoDone:
                break
            if W == 0 and stratsEvoDone:
                break
    return {"graph":graph, "history":pd.DataFrame(history, dtype="category"), "initStrats":initialStrats, "finalStrats":deepcopy(strats), "initMedStrats":initialMedStrats,  "medStrats":deepcopy(medStrats)}

# Initializes strats, payoffs, history, graph and runs many evolution episodes, each for a random node x
# runEvolution :: int -> graph -> [strat] -> [float] -> [[strat]]
runEvolution = genericRunEvolution$(runBaseEvolutionEp)
runEvolutionCompetition = genericRunEvolution$(runEvolutionCompetitionEp)

# Runs evolution, times it
def runExperiment(N=_N, episode_n=_episode_n, W=_W, graph=None, ts=(_T,_S), strats=None, beta = 0.005, k=None, useMediator=useNoMed, history=None, saveHistory=False):
    start = time()
    strats = deepcopy(strats) if strats else initStrats(N)
    experimentResults = runEvolution(N, episode_n, W, makeTSDilemma(*ts), deepcopy(strats), beta, deepcopy(graph), k, useMediator, history, saveHistory=False)
    end = time()
    print(f'running experiment N={N}, epN={episode_n} W={W}, ts={ts}, beta={beta}, k={k} took {floor(end-start)}s')
    return experimentResults

def runCompetitionExperiment(N=_N, episode_n=_episode_n, W=_W, W2=_W2, graph=None, ts=(_T,_S), medStrats=None, strats=None, beta = 0.005, k=None, medSet=_medSet, history=None, saveHistory=False):
    start = time()
    strats = deepcopy(strats) if strats else initStrats(N)
    medStrats = deepcopy(medStrats) if medStrats else initMedStrats(N,medSet)
    experimentResults = runEvolutionCompetition(N, episode_n, W, W2, makeTSDilemma(*ts), deepcopy(medStrats), deepcopy(strats), beta, deepcopy(graph), k, history, saveHistory=saveHistory)
    end = time()
    print(f'running experiment N={N}, epN={episode_n} W={W}, ts={ts}, beta={beta}, k={k} took {floor(end-start)}s')
    return experimentResults

# Runs one experiment for each game configuration in a MxM matrix os T,S values
def runTSExperiment(M, N=_N, episode_n=_episode_n, W=_W, graph=None, strats=None, beta = 0.005, k=None, useMediator=useNoMed, saveHistory=False):
    start = time()
    strats = deepcopy(strats) if strats else initStrats(N)
    ts = genTSParams(M)
    results = {(t,s):runEvolution(N=N, episode_n=episode_n, W=W, dilemma=makeTSDilemma(t,s), strats=deepcopy(strats), beta=beta, _graph=deepcopy(graph), k=k, useMediator=useMediator, saveHistory=(True if (t,s)==(2,-1) else False)) for t,s in ts}
    end = time()
    print(f'running t,s experiment N={N}, epN={episode_n} W={W}, beta={beta} took {floor(end-start)}s')
    return results

# Runs many experiments, one for each value in argDict
def runManyExperiments(argDict):
    return {key: runExperiment(**args) for key,args in argDict.items()}
# Runs many TS experiments, one for each value in argDict
def runManyTSExperiments(argDict):
    return {key: runTSExperiment(**args) for key,args in argDict.items()}

#deprecated
def runWExperiments(ws, M=9, N=100, episode_n=2, graph=None, beta = 0.005):
    return {w: runTSExperiment(M, N, episode_n=episode_n, W=w, graph=graph) for w in ws}
#deprecated
def runMediatorExperiments(meds, M=9, N=100, episode_n=2, graph=None, strats=None, k=None, beta = 0.001):
    return {useMediator.__name__: runTSExperiment(M, N, episode_n=episode_n, W=w, graph=graph, useMediator=useMediator) for useMediator in meds}


In [19]:
def continueTSExperiment(args, res):
    prevGraphs = {k:v['graph'] for k,v in res.items()}
    prevStrats = {k:v['finalStrats'] for k,v in res.items()}
    prevHistories = {k:v['history'] for k,v in res.items()}
    start = time()
    print(f'running t,s experiment N={N}, epN={episode_n} W={W}, beta={beta}')
    ts = genTSParams(M)
    results = {(t,s):runEvolution(N=args['N'], episode_n=args['episode_n'], W=args['W'], dilemma=makeTSDilemma(t,s), strats=deepcopy(prevStrats[(t,s)]), beta=args['beta'], graph=deepcopy(prevGraphs[(t,s)]), k=args['k'], useMediator=args['useMediator'], history=deepcopy(prevHistories[(t,s)])) for t,s in ts}
    end = time()
    print(f'running t,s experiment N={N}, epN={episode_n} W={W}, beta={beta} took {floor(end-start)}s')
    return results

In [20]:
# Plots
def drawGraph(graph, strats=None, mplfig=None):
    strats = strats if strats else [None for i in range(graph.num_vertices())] 
    strat2Color = {C:'green', D:'red', None: 'gray'}
    color = graph.new_vertex_property("string")
    for i,strat in enumerate(strats):
        color[graph.vertex(i)] = strat2Color[strat]#
    if mplfig:
        print('drawing graph as subplot')
        graph_draw(graph, bg_color="white", vertex_fill_color=color, mplfig=mplfig) 
    else:
        graph_draw(graph, bg_color="white", vertex_fill_color=color) 
        
def edgeStratColor(e):
    c = (strats[int(e.target())]==C) + (strats[int(e.source())]==C)
    return {0:'red', 1:'orange', 2:'green'}[c]

# plot graph with colored edges 
def plotGraph2(n, graph, strats):
    vIds = graph.new_vertex_property("string")
    stratColors = graph.new_vertex_property("string")
    edgeColors = graph.new_edge_property("string")
    for i in range(n):
        stratColors[i] = {C:'green', D:'red'}[strats[i]]
        vIds[i] = f'{i}'
    for e in graph.edges():
        edgeColors[e] = edgeStratColor(e)
    deg = graph.degree_property_map("out")
    deg.a = 25 * (np.sqrt(deg.a/n))+3
    pos = fruchterman_reingold_layout(graph)
    graph_draw(graph,pos=pos, vertex_text=vIds, vertex_font_size=16, vertex_fill_color=stratColors, vertex_size=deg, edge_color=edgeColors)
    
def plotHist(_list, bins) = pd.Series(_list).plot.hist(bins = bins)



# plot for different initial K, for different beta
def finalCoopsByW(res, game=(2.0,-1.0)) = pd.DataFrame({"W":manyTsRes.keys(), "cooperators":[Counter(manyTsRes[w][game]['episodes'][-1])[C] for w in manyTsRes.keys()]}).plot.line(x="W", y="cooperators")
def finalKMaxByW(res):
    return


def plotDegreeLog(yFn, graph, title='', xlabel="$k$", ylabel="$NP(k)$"):
    hist = gt.stats.vertex_hist(graph, 'out')
    y = yFn(hist[0])
    err = np.sqrt(y)
    err[err >= y] = y[err >= y] - 1e-2
    plt.plot(hist[1][:-1], y, "o", label="degree")
#     plt.errorbar(hist[1][:-1], y, fmt="o", yerr=err,label="in")
    plt.xlabel(xlabel)
    plt.ylabel("$NP(k)$")
    plt.tight_layout()
    ax = plt.gca()
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_ylim(1e-3, 1.5)
    ax.set_xlim(0.8, 1e3)
    return ax
def plotDD(graph): 
    return plotDegreeLog(y->y/graph.num_vertices(), graph)
def cumulativeSum(y, n) = np.sum(y[n:])
def plotCDD(graph):
    return plotDegreeLog(y->np.flip(np.cumsum(np.flip(y)))/graph.num_vertices(), graph, ylabel="$D(k)$")

def avgSquares(graph) = 
    counts,degrees = gt.stats.vertex_hist(graph,'out')
    np.sum([(degrees[i]**2)*counts[i] for i in range(len(counts))])/graph.num_vertices()
def squaredAvg(graph) = 
    gt.stats.vertex_average(graph,'out')[0]**2

def heterogeneity(graph) = avgSquares(graph) - squaredAvg(graph)
    
def plotLandscape(ts, vals, axis=None, valName=''):
    size = 4
    M = len(vals)
    df = pd.DataFrame(zip(vals, *transposeList(ts)), columns=[valName, 't', 's'])
    df = df.pivot('s', 't', valName).iloc[::-1]
    if not axis:
        fig, ax = plt.subplots(figsize=(size+1,size))         # Sample figsize in inches
        ax = sns.heatmap(df, annot=True, cbar=True, xticklabels=2, yticklabels=2, ax=ax)#.set_title(title)
        #  ax = sns.heatmap(df, annot=False, cbar=True, xticklabels=2, yticklabels=2)#.set_title(title)
        plt.show()
    else:
        ax = sns.heatmap(df, annot=False, cbar=True, xticklabels=2, yticklabels=2, ax=axis)#.set_title(title)
    return ax

def coopCount(finalStrats):
    return Counter(finalStrats)['C']
def coopLandscape(ts, res, title='', axis=None):
    cCounts = [coopCount(r['finalStrats']) for r in res]
    return plotLandscape(ts, cCounts, axis=axis, valName='coop counts')

def heterogeneityLandscape(ts, res, title='', axis=None):
    hVals = [heterogeneity(r['graph']) for r in res]
    return plotLandscape(ts, hVals, axis=axis, valName='heterogeneity')

def maxDegreeLandscape(ts, res, title='', axis=None):
    hVals = [maxDegree(r['graph']) for r in res]
    return plotLandscape(ts, hVals, axis=axis, valName='max degree (k)')


#  cCounts = [Counter(r['episodes'][-1])['C'] for r in res]
#     M = len(cCounts)
#     df = pd.DataFrame(zip(cCounts, *transposeList(ts)), columns=['count', 't', 's'])
#     df = df.pivot('s', 't', 'count').iloc[::-1]
#     if not axis:
#         ax = sns.heatmap(df, annot=False, cbar=True, xticklabels=2, yticklabels=2)#.set_title(title)
#         plt.show()
#     else:
#         ax = sns.heatmap(df, annot=False, cbar=True, xticklabels=2, yticklabels=2, ax=axis)#.set_title(title)
#     return ax

def hist2StratCount(old, new):
    d = {D:{C:1, D:0},  C:{C:0,D:-1}}
    return d[old][new]

def historyToStratCounts(initStrat, history):
    N = len(initStrat)
    initC=Counter(initStrat)[C]
    steps = history |> filter$(x->x["updateType"]=="strat") |> map$(x -> hist2StratCount(x['old'], x['new'])) |>list
    Cs = np.cumsum(steps)+initC
    Ds = map(c->N-c, Cs)
    return [{C:c, D:d} for c,d in zip(Cs,Ds)]

def plotStratEvo(initStrat, history, nSteps=500):
    stratCounts = historyToStratCounts(initStrat, history) 
    stepSize = int(max(np.floor(len(stratCounts)/nSteps), 1))
    pd.DataFrame(stratCounts[::stepSize]).plot.line(color={C:'orange', D:'blue'})
    
def dfStratCounts(run):
    stratSet = [C,D]
    initCounts = Counter(run["initStrats"])
    N=len(run["initStrats"])
    dfHist = pd.DataFrame(run['history'])
    dfStratHist = dfHist[dfHist["updateType"]=="strat"].drop(columns=["updateType"])
    for strat in stratSet:
        colName = strat+"_delta"
        dfStratHist[colName] = 0
        dfStratHist.loc[dfStratHist['new'].values==strat, colName] += 1 
        dfStratHist.loc[dfStratHist['old'].values==strat, colName] -= 1 
    dfStratHist = dfStratHist[dfStratHist.columns.difference(['old', 'new'])]
    dfStratCounts=dfStratHist[[strat+"_delta" for strat in stratSet]].cumsum()
    for strat in stratSet:
        dfStratCounts[strat+"_delta"] += initCounts[strat]
    dfStratCounts = dfStratCounts.rename(columns={strat+"_delta":strat+"_count" for strat in stratSet})
    return dfStratCounts/N

def plotStratCounts(run):
    stratCounts = dfStratCounts(run)
    stratCounts.plot.line(color={"C_count":'orange', "D_count":'blue'})

In [21]:
'''Simplex plots'''
import ternary

    
def simplexPath(episodes, title=""):
    ## Sample trajectory plot
    figure, tax = ternary.figure(scale=1.0)
    figure.set_size_inches(15, 15, forward=True)
    tax.boundary()
    tax.gridlines(multiple=0.2, color="black")
    tax.set_title(title, fontsize=20)
    points = []
    # Plot the data
    tax.plot(episodes, linewidth=2.0, label="Curve")
    tax.ticks(axis='lbr', multiple=0.2, linewidth=1, tick_formats="%.1f")
    tax.legend()
    tax.show()

def simplexPaths(runs, title=''):#
    fontsize=10
    figure, tax = ternary.figure(scale=1.0)
    figure.set_size_inches(15, 15, forward=True)
    tax.boundary()
    tax.gridlines(multiple=0.2, color="black")
    tax.set_title(title, fontsize=20, y=1.08)
    points = []
    tax.right_corner_label(NO_MED, fontsize=fontsize)
    tax.top_corner_label(GOOD_MED, fontsize=fontsize)
    tax.left_corner_label(FAIR_MED, fontsize=fontsize)

    # Plot the data
    for run in runs:
        tax.plot(run, linewidth=2.0, label="Curve")
    tax.ticks(axis='lbr', multiple=0.2, linewidth=1, tick_formats="%.1f")
    tax.legend()
    tax.show()
    

def medPropSimplex(runs, title=''):
    vecRuns = map(map$(makePropVector3), runs)
    simplexPaths(vecRuns,title)
    return

def dfMedDeltas(run, medSet=None):
    if medSet == None:
        initCounts = Counter(run["initMedStrats"])
        medSet = sorted(initCounts.keys())
    dfHist = pd.DataFrame(run['history'])
    dfMedHist = dfHist[dfHist["updateType"]=="mediator"].drop(columns=["updateType"])
    for med in medSet:
        colName = med+"_delta"
        dfMedHist[colName] = 0
        dfMedHist.loc[dfMedHist['new'].values==med, colName] += 1 
        dfMedHist.loc[dfMedHist['old'].values==med, colName] -= 1 
    return dfMedHist[dfMedHist.columns.difference(['old', 'new'])]
def dfMedCounts(run):
    initCounts = Counter(run["initMedStrats"])
    medSet = sorted(initCounts.keys())
    N=len(run["initMedStrats"])
    dfMedHist = dfMedDeltas(run, medSet)
    dfMedCounts=dfMedHist[[medName+"_delta" for medName in medSet]].cumsum()
    for med in medSet:
        dfMedCounts[med+"_delta"] += initCounts[med]
    dfMedCounts = dfMedCounts.rename(columns={med+"_delta":med+"_count" for med in medSet})
    return dfMedCounts/N

def plotMedEvolution(run, nSteps=500):
    stepSize = int(max(np.floor(len(run['initMedStrats'])/nSteps), 1))
    return simplexPath(dfMedCounts(run)[::stepSize].to_numpy())

def plotMedEvolutions(runs, nSteps=500):
    stepSize = int(max(np.floor(len(runs[0]['initMedStrats'])/nSteps), 1))
    return simplexPaths([dfMedCounts(run)[::stepSize].to_numpy() for run in runs], "")

def plotMedEvolutions(runs, nSteps=500):
    stepSize = int(max(np.floor(len(runs[0]['initMedStrats'])/nSteps), 1))
    return simplexPaths([dfMedCounts(run)[::stepSize].to_numpy() for run in runs], "")


In [22]:
def plotKeysCoopByW(unif_res)=pd.DataFrame({key: [Counter(x['episodes'][-1])[C] for w,x in res.items()] for key, res in unif_res.items()}, index=list(unif_res.values())[0].keys()).plot.line()

def plotKeysMaxKByW(unif_res)=pd.DataFrame({key: [maxDegree(x['graph']) for w,x in res.items()] for key, res in unif_res.items()}, index=list(unif_res.values())[0].keys()).plot.line()

def plotBetasCoopByW(unif_res)=pd.DataFrame({beta: [Counter(x['episodes'][-1])[C] for w,x in res.items()] for beta, res in unif_res.items()}, index=list(unif_res.values())[0].keys()).plot.line()

def plotBetasMaxKByW(unif_res)=pd.DataFrame({beta: [maxDegree(x['graph']) for w,x in res.items()] for beta, res in unif_res.items()}, index=list(unif_res.values())[0].keys()).plot.line()

def maxDegree(graph) = np.max(graph.get_out_degrees(graph.get_vertices()))


In [23]:



# def hist2MedCount(medSet, old, new):
#     return [1*(med==new) - 1*(med==old) for med in medSet]

# def historyToMedCounts(initMedStrats, history):
#     N = len(initMedStrats)
#     initialMedStrats = dict(Counter(initMedStrats))
#     initCounts = np.array(initialMedStrats.values()|>list)
#     medSet = sorted(initialMedStrats.keys())
#     medSteps = history |> filter$(x->x["updateType"]=="mediator") |> map$(x -> hist2MedCount(medSet, x['old'], x['new'])) |> list
#     medCounts = np.transpose(np.cumsum(np.array(medSteps), 0)+initCounts)
#     medStratEvolution = {med:medCounts[i]/N for i,med in enumerate(medSet)}
#     return medStratEvolution


    

In [24]:
# Multi Plots
def graphBeforeAfter(graph0, strats0, graph1 ,strats1): # graphs don't show up if we pass them the axis
    size=4
    xn = 2
    yn = 1
    fig, ax = plt.subplots(yn, xn, figsize=(yn*size,xn*size))
    drawGraph(graph0, strats=strats0, mplfig=ax[0])
    drawGraph(graph1, strats=strats1, mplfig=ax[1])
    plt.show()

def plot1D(res, plotFn, title='', keyName=''):
    size = 4
    items = res.items()
    n = len(items)
    fig, ax = plt.subplots(1, n, figsize=((n+1)*size,size))
    fig.suptitle(title, fontsize=16, y=1.08)
    for i,(k,v) in enumerate(items):
        plt.subplot(1,n,i+1)
        ax = plotFn(k,v, keyName)
        
def plot2D(M, res, plotFn, title='', keyName=''):
    size = 4
    items = res.items()
    n = len(items)
    fig, ax = plt.subplots(M, M, figsize=(M*size,M*size))
    fig.suptitle(title, fontsize=16, y=1.08)
    for i,(k,v) in enumerate(items):
        plt.subplot(M,M,M*M-i)
        ax = plotFn(k,v, keyName)

def plotDDFn(k,v, keyName=''):
#     graph, totalPayoffs, episodes = v
    ax = plotDD(v['graph'])
    ax.set_title(f'DD {keyName}={k}')
    return ax
def plotCDDFn(k,v, keyName=''):
    ax = plotCDD(v['graph'])
    ax.set_title(f'CDD {keyName}={k}')
    return ax

def plotStratEvoFn(k,v, keyName=''):
#     graph, totalPayoffs, episodes = v
    stratCounts = historyToStratCounts(v['initStrats'], v['history'])
    ax = pd.DataFrame(stratCounts).plot(color={C:'orange', D:'blue'}, ax=plt.gca())
    ax.set_title(f'{keyName}={k}')
    return ax

def plotHistFn(k,v, keyName=''):
#     graph, totalPayoffs, episodes = v
    ax = plotHist(*gt.stats.vertex_hist(v['graph'], 'out'))
    return ax

def plotCoopLandscapeFn(k,v, keyName=''):
#     ts,res = v
#     ax = coopLandscape(v['ts'], v['results'], title='', axis=plt.gca())
    ax = coopLandscape(v.keys(), v.values(), title='', axis=plt.gca())
    ax.set_title(f'{keyName}={k}')
    return ax

def plotHetLandscapeFn(k,v, keyName=''):
#     ts,res = v
#     ax = heterogeneityLandscape(v['ts'], v['results'], title='', axis=plt.gca())
    ax = heterogeneityLandscape(v.keys(), v.values(), title='', axis=plt.gca())
    ax.set_title(f'{keyName}={k}')
    return ax
def plotMaxDegreeLandscapeFn(k,v, keyName=''):
    ax = maxDegreeLandscape(v.keys(), v.values(), title='', axis=plt.gca())
    ax.set_title(f'{keyName}={k}')
    return ax

# for i in range(M):
#     plot2D(M{ts[M*i+j]:res for j,res in enumerate(results[M*i:M*(i+1)])}, plotStratEvoFn, keyName='t,s')
def plotStratMatrix(tsExpRes):
    _results = orderTsMatrixPlot(list(tsExpRes.values()), M)
#     _results = tsExpRes.values()
#     _ts = tsExpRes.keys()
    _ts = orderTsMatrixPlot(list(tsExpRes.keys()), M)
    tsRes = {ts:res for ts,res in reversed(zip(_ts, _results))}
#     tsRes = {ts:res for ts,res in zip(_ts, _results)}
    return plot2D(M, tsRes, plotStratEvoFn, keyName='t,s')

def plotCDDMatrix(tsExpRes):
    _results = orderTsMatrixPlot(list(tsExpRes.values()), M)
#     _results = tsExpRes.values()
#     _ts = tsExpRes.keys()
    _ts = orderTsMatrixPlot(list(tsExpRes.keys()), M)
    tsRes = {ts:res for ts,res in reversed(zip(_ts, _results))}
#     tsRes = {ts:res for ts,res in zip(_ts, _results)}
    return plot2D(M, tsRes, plotCDDFn, keyName='t,s')


In [25]:
def countUpdateTypes(history):
    updateTypes = history |> map$(.["updateType"])
    return Counter(updateTypes) |> dict

def nEmptyRewires(history):
    return history |> filter$(x->x["updateType"]=="rewire" and x['old'] == x['new']) |> list |> len
def nEmptyStratChanges(history):
    return history |> filter$(x->x["updateType"]=="strat" and x['old'] == x['new']) |> list |>  len

def historyStats(history):
    print(f'countUpdateTypes: {countUpdateTypes(history)}')
    print(f'nEmptyRewires: {nEmptyRewires(history)}')
    print(f'nEmptyStratChanges: {nEmptyStratChanges(history)}')
    

def coopCountsRes(res):
    gameParams = list(res[0].keys())
    coopCounts = {game:list(map(x->coopCount(x[game]['finalStrats']) , res)) for game in gameParams}
    return coopCounts

def fractionsAbsorbCoop(N, res, absorbFraction = 0.90):
    
    nSimulations = len(res)
    coopCounts = coopCountsRes(res)
    gameParams = list(res[0].keys())
    fractions = {game:np.count_nonzero(np.array(coopCounts[game])>=N*absorbFraction)/nSimulations for game in gameParams}
    return fractions 

def avgFractionCoop(N, res):
#     d = max(1, N/1e4)
    d = 1
    coopCounts = coopCountsRes(res)
    gameParams = list(res[0].keys())
    avgs = {game:np.mean(coopCounts[game])/N for game in gameParams}
    return avgs

    
def pickKeyMany(key, obj):
    return map$(obj[key])

def finalExperiment():
    # 100 simulations for each set of parameters (T,S,W)
    absorbFraction = 0.90 
    nSimulations = 10
    for w in ws:
        args['W'] = w
        res = [runTSExperiment(**args) for i in range(nSimulations)]
        # compute fraction of times that evolution stopped at 100% cooperation
        coopCounts = coopCountsRes(res)
        absorbFractions = fractionsAbsorbCoop(N, res, absorbFraction)
        # if no convergence after 10^8 generations
        avgCoopFractions = avgFractionCoop(N,res)
        plotColor = {game:absorbFractions[game] if absorbFractions[game] != 0 else avgCoopFractions[game] for game in gameParams}
        
    # take avg fraction of cooperators over the last 1000 generations
    

In [26]:
# Save to files
def pToStr(param):
    return f"{param[0]}-{param[-1]}" if isinstance(param, list) else str(param)
def makePath(name):
    return f"./data/{name}.pkl"
def renameDuplicate(makePath, name, i=0):
    print(f"renameDuplicate {i}")
    _name = deepcopy(name)
    if i != 0:
        _name = f'{_name} ({i})'
    if os.path.exists(makePath(_name)):
        return renameDuplicate(makePath, name, i+1)
    return _name 
def makeExperimentName(useMediator, N,M,episode_n,beta,W,k):
    baseName = f"{useMediator.__name__}_N-{pToStr(N)}_M-{pToStr(M)}_episoden-{pToStr(episode_n)}_beta-{pToStr(beta)}_W-{pToStr(W)}_k-{pToStr(k)}"
    return baseName
def saveRes(res, _name):
    name = renameDuplicate(makePath, _name)
    os.makedirs('./data', exist_ok=True)
    path = makePath(name)
    with open(path, "wb+") as file:
        pickle.dump(res, file)
        print(f"saved {name}.pkl")
def loadRes(name):
    with open(f"./data/{name}.pkl", "rb") as file:
        res = pickle.load(file)
        print(f"loaded {name}.pkl")
        return res 
def loadResFn(filename):
    with open(f"./data/{filename}", "rb") as file:
        res = pickle.load(file)
        print(f"loaded {filename}")
        return res 

In [27]:
def makeCompetitionName(medSet, N, episode_n,ts,beta,W1,W2,k):
    baseName = f"{'-'.join(medSet)}_N-{pToStr(N)}_episoden-{pToStr(episode_n)}_beta-{pToStr(beta)}_W1-{pToStr(W1)}_W2-{pToStr(W2)}_k-{pToStr(k)}"
    return baseName

# Mediator competition with rewiring, no game strat evolution 

In [31]:
n_trials = 2
# _medSet = [NO_MED, GOOD_MED, FAIR_MED, RANDOM_MED, BAD_MED]
medSet = [NO_MED, FAIR_MED]
N=500
episode_n=10000
W1=1
W2=1
ts=(2,-1)
beta=0.005
k=30
runs = [runCompetitionExperiment(N=N, episode_n=episode_n, W=W1, W2=W2, ts=ts, beta=beta, k=k, saveHistory=True, history=None, medSet=medSet) for i in range(n_trials)]
# run = runCompetitionExperiment(N=500, episode_n=100000, W=0, saveHistory=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=10000
running experiment N=500, epN=10000 W=1, ts=(2, -1), beta=0.005, k=30 took 1s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=10000
running experiment N=500, epN=10000 W=1, ts=(2, -1), beta=0.005, k=30 took 1s


In [78]:
saveRes(runs, makeCompetitionName(medSet, N, episode_n,ts,beta,W1,W2,k))

renameDuplicate 0
saved NO_MED-FAIR_MED_N-500_episoden-10000_beta-0.005_W1-1_W2-inf_k-30.pkl


In [29]:
medSet = [NO_MED, FAIR_MED]
def competitionNoStratEvo(medSet, save=False):
    n_trials = 5
    N=500
    episode_n=1000000
    W1=1
    W2=inf
    ts=(2,-1)
    beta=0.005
    k=30
    runs = [runCompetitionExperiment(N=N, episode_n=episode_n, W=W1, W2=W2, ts=ts, beta=beta, k=k, saveHistory=True, history=[], medSet=medSet) for i in range(n_trials)]
    if save:
        saveRes(runs, makeCompetitionName(medSet, N, episode_n,ts,beta,W1,W2,k))
    return runs

## 2 by 2

### Compete with no med

In [66]:
for adv in [GOOD_MED, BAD_MED, FAIR_MED, RANDOM_MED]:
    competitionNoStratEvo([NO_MED, adv], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 125s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 105s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 105s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 109s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 106s
renameDuplicate 0
saved NO_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running e

### Compete with random

In [28]:
for adv in [GOOD_MED, BAD_MED, FAIR_MED]:
    competitionNoStratEvo([RANDOM_MED, adv], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 96s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 100s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 109s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 110s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 96s
renameDuplicate 0
renameDuplicate 1
saved RANDOM_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30 (1).pkl
starting runEvolution, history len= 0, saving history,  N=500, epi

### Compete with fair

In [29]:
for adv in [GOOD_MED, BAD_MED]:
    competitionNoStratEvo([FAIR_MED, adv], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 94s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 97s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 100s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 98s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 99s
renameDuplicate 0
saved FAIR_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running exp

In [113]:
runs2by2 = {fname: loadResFn("2by2/"+fname) for fname in os.listdir("./data/2by2")}

loaded 2by2/FAIR_MED-BAD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/FAIR_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/NO_MED-BAD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/NO_MED-FAIR_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/NO_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/NO_MED-RANDOM_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/RANDOM_MED-BAD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/RANDOM_MED-FAIR_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
loaded 2by2/RANDOM_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl


In [28]:
fname = ipywidgets.Select(
    options=os.listdir("./data/full evo/complete run/2by2"),
    rows=20,
    description='run:',
    disabled=False,
)

# fname = ipywidgets.Select(
#     options=runs2by2.keys(),
#     rows=10,
#     description='run:',
#     disabled=False,
# )

def plotRes(fname):
    runs = loadResFn("full evo/complete run/2by2/"+fname)
    for run in runs:
        medcounts = dfMedCounts(run)
        medcounts.plot()
    
# def plotRes(fname):
#     runs = runs2by2[fname]
#     medcounts = dfMedCounts(runs[0])
#     medcounts.plot()

ipywidgets.interact(plotRes, fname=fname)

interactive(children=(Select(description='run:', options=('FAIR_MED-BAD_MED_N-500_episoden-1000000_beta-0.005_…

<function __main__.plotRes(fname)>

## 3 by 3

In [36]:
from itertools import combinations

In [37]:
for advs in combinations([RANDOM_MED, GOOD_MED, BAD_MED, FAIR_MED], 2):
    competitionNoStratEvo([NO_MED, *advs], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 96s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 62s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 33s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 20s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 71s
renameDuplicate 0
saved NO_MED-RANDOM_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-inf_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
run

In [31]:
fname = ipywidgets.Select(
    options=os.listdir("./data/3by3"),
#     value='OSX',
    rows=20,
    description='OS:',
    disabled=False,
)

def plotRes(fname):
    runs = loadResFn("3by3/"+fname)
    medcounts = dfMedCounts(runs[0])
    medcounts.plot()

ipywidgets.interact(plotRes, fname=fname)

interactive(children=(Select(description='OS:', options=('NO_MED-BAD_MED-FAIR_MED_N-500_episoden-1000000_beta-…

<function __main__.plotRes(fname)>

## Interactive

In [35]:
subdir = "2by2"
fname2 = ipywidgets.Select(
    options=os.listdir("./data/"+subdir),
    rows=20,
    description='OS:',
    disabled=False,
)

def plotRes(fname):
    runs = loadResFn(f'{subdir}/{fname}')
    medcounts = dfMedCounts(runs[0])
    medcounts.plot()

ipywidgets.interact(plotRes, fname=fname2)

interactive(children=(Select(description='OS:', options=('FAIR_MED-BAD_MED_N-500_episoden-1000000_beta-0.005_W…

<function __main__.plotRes(fname)>

In [ ]:
fname3 = ipywidgets.Select(
    options=os.listdir("./data/3by3"),
    rows=20,
    description='OS:',
    disabled=False,
)


ipywidgets.interact(plotRes, fname=fname)

In [124]:
def dfMedDeltas(run, medSet):
    dfHist = pd.DataFrame(run['history'])
    dfMedHist = dfHist[dfHist["updateType"]=="mediator"].drop(columns=["updateType"])
    doMedCount=x-> 1*(med==x['new']) - 1*(med==x['old'])
    for med in medSet:
        dfMedHist[med+"_delta"] = dfMedHist.apply(doMedCount, axis=1)
    return dfMedHist


In [186]:
def dfMedDeltas(run, medSet):
    dfHist = pd.DataFrame(run['history'])
    dfMedHist = dfHist[dfHist["updateType"]=="mediator"].drop(columns=["updateType"])
    for med in medSet:
        colName = med+"_delta"
        dfMedHist[colName] = 0
        dfMedHist.loc[dfMedHist['new'].values==med, colName] += 1 
        dfMedHist.loc[dfMedHist['old'].values==med, colName] -= 1 
    return dfMedHist
def dfMedCounts(run):
    initCounts = Counter(run["initMedStrats"])
    medSet = sorted(initCounts.keys())
    N=len(run["initMedStrats"])
    dfMedHist = dfMedDeltas(run, medSet)
    dfMedCounts=dfMedHist[[medName+"_delta" for medName in medSet]].cumsum()
    for med in medSet:
        dfMedCounts[med+"_delta"] += initCounts[med]
    dfMedCounts = dfMedCounts.rename(columns={med+"_delta":med+"_count" for med in medSet})
    return dfMedCounts/N

Winners still seem pretty random: 2 random, 2 good, 1 fair, 1 bad 

# Med competition with rewiring and strats

In [35]:
def competitionFullEvo(medSet, save=False):
    n_trials = 5
    N=500
    episode_n=10000
    W1=1
    W2=1
    ts=(2,-1)
    beta=0.005
    k=30
    runs = [runCompetitionExperiment(N=N, episode_n=episode_n, W=W1, W2=W2, ts=ts, beta=beta, k=k, saveHistory=True, history=[], medSet=medSet) for i in range(n_trials)]
    if save:
        saveRes(runs, makeCompetitionName(medSet, N, episode_n,ts,beta,W1,W2,k))
    return runs

## 2 by 2

### Compete with no med

In [28]:
for adv in [GOOD_MED, BAD_MED, FAIR_MED, RANDOM_MED]:
    competitionFullEvo([NO_MED, adv], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 56s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 36s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 78s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 126s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 126s
renameDuplicate 0
saved NO_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-1_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experi

### Compete with random

In [29]:
for adv in [GOOD_MED, BAD_MED, FAIR_MED]:
    competitionFullEvo([RANDOM_MED, adv], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 126s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 134s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 128s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 128s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 126s
renameDuplicate 0
saved RANDOM_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-1_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running

### Compete with fair

In [30]:
for adv in [GOOD_MED, BAD_MED]:
    competitionFullEvo([FAIR_MED, adv], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 103s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 123s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 101s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 129s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 52s
renameDuplicate 0
saved FAIR_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-1_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running ex

In [27]:
fname = ipywidgets.Select(
    options=os.listdir("./data/2by2"),
    rows=20,
    description='run:',
    disabled=False,
)

# fname = ipywidgets.Select(
#     options=runs2by2.keys(),
#     rows=10,
#     description='run:',
#     disabled=False,
# )

def plotRes(fname):
    runs = loadResFn("2by2/"+fname)
    for run in runs:
        medcounts = dfMedCounts(run)
        medcounts.plot()
    
# def plotRes(fname):
#     runs = runs2by2[fname]
#     medcounts = dfMedCounts(runs[0])
#     medcounts.plot()

ipywidgets.interact(plotRes, fname=fname)

FileNotFoundError: [Errno 2] No such file or directory: './data/2by2'

## 3 by 3

In [31]:
from itertools import combinations

In [32]:
for advs in combinations([RANDOM_MED, GOOD_MED, BAD_MED, FAIR_MED], 2):
    competitionFullEvo([NO_MED, *advs], save=True)

starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 123s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 48s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 128s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 88s
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
running experiment N=500, epN=1000000 W=1, ts=(2, -1), beta=0.005, k=30 took 120s
renameDuplicate 0
saved NO_MED-RANDOM_MED-GOOD_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-1_k-30.pkl
starting runEvolution, history len= 0, saving history,  N=500, episode_n=1000000
ru

In [116]:
fname = ipywidgets.Select(
    options=os.listdir("./data/full evo/complete run/3by3"),
#     value='OSX',
    rows=20,
    description='OS:',
    disabled=False,
)

def plotRes(fname):
    runs = loadResFn("full evo/complete run/3by3/"+fname)
    for run in runs:
        fig, axes = plt.subplots(nrows=1,ncols=2)
        stratCounts = dfStratCounts(run)
        stratCounts.plot(ax = axes[0], subplots=True)
        medcounts = dfMedCounts(run)
        medcounts.plot(ax = axes[1], subplots=True)

ipywidgets.interact(plotRes, fname=fname)

interactive(children=(Select(description='OS:', options=('NO_MED-BAD_MED-FAIR_MED_N-500_episoden-1000000_beta-…

<function __main__.plotRes(fname)>

In [84]:
fname = "full evo/complete run/3by3/NO_MED-BAD_MED-FAIR_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-1_k-30.pkl"
runs = loadResFn(fname)

loaded full evo/complete run/3by3/NO_MED-BAD_MED-FAIR_MED_N-500_episoden-1000000_beta-0.005_W1-1_W2-1_k-30.pkl


# Performance optimization

In [14]:
!pip install line_profiler

In [33]:
%load_ext line_profiler

In [5]:
def makeArgs():
    N = 500
    M=5
    episode_n = 1000
    graph0 = initUniformRandomGraph(N)
    beta = 0.005
    W = 0
    k=30
    args = {"N":N, "M":M, "graph":graph0, "episode_n":episode_n, "beta":beta, "W":W, "k":k}
    return args

def profFunction():
    args = makeArgs()
    res = runTSExperiment(**args)


In [16]:

# runEvolution :: int -> graph -> [strat] -> [float] -> [[strat]]
def _runEvolution(N, episode_n, W, dilemma, strats=None, beta = 0.001, graph=None, k=None, useMediator=useNoMed):
    totalPayoffs = initPayoffs(N)
    strats = deepcopy(strats) if strats else initStrats(N)
    history = [] #[{updateType: "strat", x: N, old: {C,D}, new: {C,D}}, {updateType: "rewire", xy: (x,y), old: (x,y), new: (x,z)}]
    graph = deepcopy(graph) if graph else initUniformRandomGraph(N=N, k=(k if k else _k))
    for i in range(episode_n):
        x = np.random.randint(N)
        strats, graph, history = runEvolutionEp(N, beta, W, dilemma, graph, strats, history, x, useMediator)
        if i % 10 == 0:
            counts = Counter(strats)
            if counts[C] == N or counts[D] == N: 
                break
    return {"graph":graph, "history":history, "initStrats":initStrats, "finalStrats":deepcopy(strats)}


In [24]:
%lprun -f _runEvolution _runEvolution(500, 10000, 2, makeTSDilemma(2,-1))

In [54]:
def fermi(beta, fitness_diff):
    fitness_diff = np.clip(fitness_diff, 0, None)
    big = np.exp(-1 * beta * fitness_diff, dtype=np.float64)
    return 1. / (1. + big)
    
%lprun -f fermi fermi(0.005, np.random.randint(-20, 20))

In [76]:
def makeStructArgs():
    N=500
    graph0 = initUniformRandomGraph(N)
    beta = 0.005
    dilemma=makeTSDilemma(2,-1)
    strats=initStrats(N)
    useMediator=useNoMed
    x=np.random.randint(1, N-1)
    y=np.random.randint(1, N-1)
    args={"_x":x, "_y":y, "p":random(), "graph":graph0, "beta":beta,"dilemma":dilemma, "strats":strats, "useMediator":useMediator}
    return args

def _calcStructuralUpdate(beta, graph, dilemma, strats, _x, _y, p, useMediator):
    x,y = int(_x), int(_y)
    sx, sy = strats[x],strats[y]
    res = TieUpdate(x,y,y)
    if (sx == C and sy == D):
        doRewire = random() < p        
        if doRewire:
            res = decideRewire(graph, strats, x, y, useMediator)
        else:
            res = TieUpdate(x,y,y)
    elif (sx == D and sy == D):
        keepX = random() < p
        args = [x,y] if keepX else [y,x]
        res = decideRewire(graph, strats, args[0], args[1], useMediator)
    return res


In [79]:
%lprun -f _calcStructuralUpdate _calcStructuralUpdate(**makeStructArgs())

In [80]:
def makeRewireArgs():
    N=500
    graph0 = initUniformRandomGraph(N)
    beta = 0.005
    strats=initStrats(N)
    useMediator=useNoMed
    x=np.random.randint(N)
    y=np.random.randint(N)
    args={"x":x, "y":y, "graph":graph0, "strats":strats, "useMediator":useMediator}
    return args

def _decideRewire(graph, strats, x, y, useMediator):
    # if isLonely(graph, x) or isLonely(graph, y): 
    if isLonely(graph, y): 
        tieUp = TieUpdate(x,y,y) #enforcing graph connectedness
        return tieUp
    # z = sampleNeighborUnique(graph, y, x)
    z = useMediator(graph, strats, y, x)
    if not z:
        tieUp = TieUpdate(x,y,y) #enforcing graph connectedness
        return tieUp
    tieUp = TieUpdate(x,y,z) #enforcing graph connectedness
    return tieUp


In [81]:
%lprun -f _decideRewire _decideRewire(**makeRewireArgs())

In [89]:
def eligibleNewFriends(graph, b, a):
    bs = graph.get_all_neighbors(b)
    _as = graph.get_all_neighbors(a)
    x = set(bs) - set(_as) - set([a])
    xs = list(x)
    return xs

def useNoMed(graph, strats, y, x) = sampleSecondNeighborUnique(graph,x)


In [126]:
%lprun -f eligibleNewFriends eligibleNewFriends(graph, np.random.randint(N), np.random.randint(N))

In [1140]:
def makeStratArgs():
    graph0 = initUniformRandomGraph(N)
    beta = 0.005
    dilemma=makeTSDilemma(2,-1)
    strats=initStrats(N)
    x=1
    args={"x":x,"graph":graph0, "beta":beta,"dilemma":dilemma, "strats":strats}
    return args

def _calcStrategyUpdate(beta, graph, dilemma, strats, x):
    y = _sampleNeighbor(graph, x)
    _x,_y = int(x), int(y)
    px, py = pairCumPayoffs(dilemma, graph, strats, x,y)
    if px > py:
        return {"updateType":"strat", "x":_x, "old":strats[_x], "new":strats[_x]}
    p = fermi(beta, py - px)     
    doChangeStrat = random() < p  
    new = strats[_y] if doChangeStrat else strats[_x]
    return {"updateType":"strat", "x":_x, "old":strats[_x], "new":new}    

In [1157]:
%lprun -f _calcStrategyUpdate _calcStrategyUpdate(**makeStratArgs())

In [223]:
def makeCumPayoffsArgs():
    graph0 = initUniformRandomGraph(N)
    dilemma=makeTSDilemma(2,-1)
    strats=initStrats(N)
    x=1
    args={"x":x,"graph":graph0, "dilemma":dilemma, "strats":strats}
    return args
def playDilemma(dilemma, strats, id1, id2) = dilemma[strats[id1]][strats[id2]]
def _nodeCumPayoffs(dilemma, graph, strats, x):
    payoffs = [playDilemma(dilemma, strats, x, y)[0] for y in graph.get_all_neighbors(x)]
    payoff = sum(payoffs)
    return payoff



In [231]:
%lprun -f _nodeCumPayoffs _nodeCumPayoffs(**makeCumPayoffsArgs())